<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/network_LSTM/network/LSTM/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

## Basic includes

In [1]:
!pip install ipython-autotime
%load_ext autotime

time: 530 µs (started: 2022-02-20 09:22:10 +00:00)


In [2]:
!pip install word2vec

     |████████████████████████████████| 42 kB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for word2vec: filename=word2vec-0.11.1-py2.py3-none-any.whl size=156420 sha256=9783e4fe371713333c119cbb4ca8d521b647ea2dedf00a9ffd1e149b20969f61
  Stored in directory: /root/.cache/pip/wheels/c9/c0/d4/29d797817e268124a32b6cf8beb8b8fe87b86f099d5a049e61
Successfully built word2vec
time: 14.2 s (started: 2022-02-20 09:22:10 +00:00)


In [3]:
import word2vec
from collections import Counter # https://pymotw.com/2/collections/counter.html

import pandas as pd
import numpy as np
import itertools
import re
import os

import glob

time: 47.1 ms (started: 2022-02-20 09:22:25 +00:00)


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

dataPath = '/content/drive/MyDrive/TP2/Datasets/Recipe1M/'
import sys
sys.path.append(dataPath)

Mounted at /content/drive/
time: 1min 3s (started: 2022-02-20 09:22:25 +00:00)


## Import Data

In [5]:
TIMESTAMP = '2022_02_11'

time: 894 µs (started: 2022-02-20 09:23:28 +00:00)


In [6]:
baseFrame = pd.DataFrame()

smallSet = True

if(os.path.exists(dataPath + TIMESTAMP + '/recipes_valid_full.pkl')):
  baseFrame = pd.read_pickle(dataPath + TIMESTAMP + '/recipes_valid_full.pkl')
elif(smallSet == True):
  baseFrame = baseFrame.append(pd.read_pickle(glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl')[0]))
elif(len(glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl')) != 0):
  for file in glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl'):
    if not 'full' in file:
      baseFrame = baseFrame.append(pd.read_pickle(file))

baseFrame.head()

,title,ingredients,instructions
id,,,
000033e39b,Dilly Macaroni Salad Recipe,amount unit ingredient 0 1....,0 Cook macaroni according to package direct...
000035f7ed,Gazpacho,amount unit ingredient 0 8.0 ...,0 Add the tomatoes to a food processor with...
00003a70b1,Crunchy Onion Potato Bake,amount unit ingredient 0 2...,0 Preheat oven to 350 degrees Fah...
00004320bb,Cool 'n Easy Creamy Watermelon Pie,amount unit ingredient 0 3....,0 Dissolve Jello in boiling water. 1 ...
0000631d90,Easy Tropical Beef Skillet,amount unit ingredient 0...,"0 In a large skillet, toast the coconut ove..."


time: 28.5 s (started: 2022-02-20 09:23:28 +00:00)


In [7]:
def flatten_ing(row):
  row['amount'] = row['ingredients']['amount'].tolist()
  row['unit'] = row['ingredients']['unit'].tolist()
  row['ingredient'] = row['ingredients']['ingredient'].tolist()
  return row

baseFrame = baseFrame.apply(flatten_ing, axis=1)
baseFrame = baseFrame.drop(columns=['ingredients'])

baseFrame.head()

,title,instructions,amount,unit,ingredient
id,,,,,
000033e39b,Dilly Macaroni Salad Recipe,0 Cook macaroni according to package direct...,"[1.0, 1.0, 0.5, 0.5, 3.0, 0.5, 1.0, 0.75, 0.5]","[, , , , tablespoon, , tablespoon, teaspoon, t...","[elbow macaroni, american cheese, celery, gree..."
000035f7ed,Gazpacho,0 Add the tomatoes to a food processor with...,"[8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 3.0]","[, , , , , , , , ]","[tomatoes, kosher salt, red onion, green bell ..."
00003a70b1,Crunchy Onion Potato Bake,0 Preheat oven to 350 degrees Fah...,"[2.5, 1.5, 0.25, 1.0, 8.0, 1.0, 1.0]","[, , cup, , ounce, cup, cup]","[milk, water, butter, mashed potatoes, whole k..."
00004320bb,Cool 'n Easy Creamy Watermelon Pie,0 Dissolve Jello in boiling water. 1 ...,"[3.0, 0.25, 12.0, 2.0, 1.0]","[ounce, cup, ounce, cup, ]","[watermelon gelatin, boiling water, cool whip,..."
0000631d90,Easy Tropical Beef Skillet,"0 In a large skillet, toast the coconut ove...","[0.5, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 8.0, 16.0,...","[cup, pound-mass, tablespoon, , tablespoon, ta...","[shredded coconut, lean ground beef, fresh gar..."


time: 2min 55s (started: 2022-02-20 09:23:56 +00:00)


## Brainstorming
* not a simple sequence
* basically sequence of
  * ingredients
  * set of instructions --> instruction --> words
* maybe instead of instructions sequences, instructions could be flattened and split signs (.,!?;) also learned (in case learning sentence sequences is not possible)

Maybe stacked LSTMs are a solutions
* train how instructions in general are to be composed
* train how instructions are combined to a set of instructions

Input
* ingredients to be used as input
* to not train on instructions to appear according to ingredients sequence, maybe randomly shuffle ingredients but keep instruction set the same
* 

https://www.youtube.com/watch?v=A9QVYOBjZdY&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=5&ab_channel=TensorFlow


## Differentiation
* Title generation based on ingredients
* instruction generation

## Import Torch

In [88]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data import get_tokenizer # https://pytorch.org/text/stable/data_utils.html
from torch.utils.data import DataLoader

# torch padding does only support constant padding (ConstantPad1d) for 1D or non-constant padding for >1D (nn.function.pad)
from tensorflow.keras.preprocessing.sequence import pad_sequences
# keras tokenizer more powerful than torch
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence


torch.manual_seed(1)

time: 6.48 ms (started: 2022-02-20 15:06:01 +00:00)


In [98]:
# initialize tokenizer
# tokenizer = get_tokenizer("basic_english") # pytorch
tokenizer = Tokenizer() # keras

# to be checked: necessity of punctuation (maybe reintroduce later: https://stackoverflow.com/questions/49073673/include-punctuation-in-keras-tokenizer)

time: 1.85 ms (started: 2022-02-20 15:13:03 +00:00)


# Setup
https://closeheat.com/blog/pytorch-lstm-text-generation-tutorial

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB
time: 55.5 ms (started: 2022-02-20 09:26:57 +00:00)


### Get Corpus

In [10]:
w2v_model = word2vec.load(dataPath + 'vocab.bin')
ingredientDict = {}
for voc in w2v_model.vocab:
     # Offset by 1 so empty fields can be 0
     ingredientDict.setdefault(voc, len(ingredientDict)+1)

if 'dilly' in ingredientDict:
  print("Word exists")
else:
  print('vocab.bin not to be used as dict misses words') 

vocab.bin not to be used as dict misses words
time: 1.19 s (started: 2022-02-20 09:26:57 +00:00)


In [90]:
def getCorpus(title, ingredient, instructions):
  titleTok = text_to_word_sequence(title)
  ingTok = text_to_word_sequence(' '.join(ingredient))
  instTok = text_to_word_sequence(' '.join(instructions))
  return np.array(ingTok + titleTok + instTok)

corpus = np.vectorize(getCorpus, otypes=[np.ndarray])(baseFrame['title'], baseFrame['ingredient'], baseFrame['instructions'])
corpus = np.concatenate(corpus)

time: 8.17 s (started: 2022-02-20 15:06:09 +00:00)


###  Sequences

In [97]:
test = baseFrame.head(20).copy()

def getTitleSequence(title, ingredient):
  titleTok = text_to_word_sequence(title)
  ingTok = text_to_word_sequence(','.join(ingredient))
  return (ingTok + titleTok)

titleSeq = np.vectorize(getTitleSequence, otypes=[np.ndarray])(test['title'], test['ingredient'])
titleSeq[0]

# xs, labels = titleSeq[:,:-1],titleSeq[:,-1]

# titleSeq

['elbow',
 'macaroni',
 'american',
 'cheese',
 'celery',
 'green',
 'peppers',
 'pimentos',
 'mayonnaise',
 'vinegar',
 'salt',
 'dry',
 'dill',
 'weed',
 'dilly',
 'macaroni',
 'salad',
 'recipe']

time: 8.62 ms (started: 2022-02-20 15:12:06 +00:00)


In [94]:
def getTitleSequence(title, ingredient):
  titleTok = text_to_word_sequence(title)
  ingTok = text_to_word_sequence(','.join(ingredient))
  return {'ings': ingTok, 'title': titleTok}

titleSeq = np.vectorize(getTitleSequence, otypes=[dict])(baseFrame['title'], baseFrame['ingredient'])
titleSeq[0]['ings']

['elbow',
 'macaroni',
 'american',
 'cheese',
 'celery',
 'green',
 'peppers',
 'pimentos',
 'mayonnaise',
 'vinegar',
 'salt',
 'dry',
 'dill',
 'weed']

time: 1.74 s (started: 2022-02-20 15:06:52 +00:00)


In [95]:
def getInstSequence(ingredient, instructions):
  ingTok = text_to_word_sequence(','.join(ingredient))
  instTok = text_to_word_sequence('\n'.join(instructions))
  return np.array(ingTok + instTok)

instSeq = np.vectorize(getInstSequence, otypes=[np.ndarray])(baseFrame['ingredient'], baseFrame['instructions'])
instSeq[0]

array(['elbow', 'macaroni', 'american', 'cheese', 'celery', 'green',
       'peppers', 'pimentos', 'mayonnaise', 'vinegar', 'salt', 'dry',
       'dill', 'weed', 'cook', 'macaroni', 'according', 'to', 'package',
       'directions', 'drain', 'well', 'cold', 'combine', 'macaroni',
       'cheese', 'cubes', 'celery', 'green', 'pepper', 'and', 'pimento',
       'blend', 'together', 'mayonnaise', 'or', 'possibly', 'salad',
       'dressing', 'vinegar', 'salt', 'and', 'dill', 'weed', 'add', 'in',
       'to', 'macaroni', 'mix', 'toss', 'lightly', 'cover', 'and',
       'refrigeratewell', 'serve', 'salad', 'in', 'lettuce', 'lined',
       'bowl', 'if', 'you', 'like', 'makes', '6', 'servings'],
      dtype='<U15')

time: 5.73 s (started: 2022-02-20 15:06:55 +00:00)


### Hyperparams

In [16]:
class hyperParams():
  def __init__(self, epochs=10, batchSize=256, sequenceLength=4):
    self.epochs = epochs
    self.batchSize = batchSize
    self.sequenceLength = sequenceLength

time: 2.8 ms (started: 2022-02-20 09:27:41 +00:00)


### Padding

### Dataset

In [46]:
# Data (https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class)
class TitleDataset(torch.utils.data.Dataset):
    def __init__(self, hyperparams, data):
      self.hyperparams = hyperparams

      # dataset split into word sequences required for training
      self.wordSeq = np.vectorize(self.getTitleSequence, otypes=[np.ndarray])(data['title'], data['ingredient'])

      # list of all words in dataset
      self.words = np.concatenate(np.vectorize(self.getCorpus, otypes=[np.ndarray])(data['title'], data['ingredient'], data['instructions']))

      # list of all unique words in dataset
      self.unique = sorted(Counter(self.words), key=Counter(self.words).get, reverse=True)

      # https://closeheat.com/blog/pytorch-lstm-text-generation-tutorial
      self.index_to_word = {index: word for index, word in enumerate(unique)}
      self.word_to_index = {word: index for index, word in enumerate(unique)}

      # indexed wordSequences
      self.idxWords = np.vectorize(self.getIndexedSeqs, otypes=[np.ndarray])(self.wordSeq)

      # training requires same length sequences -->  padding

    def getCorpus(self, title, ingredient, instructions):
      titleTok = text_to_word_sequence(title)
      ingTok = text_to_word_sequence(','.join(ingredient))
      instTok = text_to_word_sequence('\n'.join(instructions))
      return np.array(ingTok + titleTok + instTok)

    def getTitleSequence(self, title, ingredient):
      titleTok = text_to_word_sequence(title)
      ingTok = text_to_word_sequence(','.join(ingredient))
      return {'ings': ingTok, 'title': titleTok}

    def getIndexedSeqs(self, seq):
      ingTok = np.vectorize(self.word_to_index.get)(seq['ings'])
      titleTok = np.vectorize(self.word_to_index.get)(seq['title'])
      return {'ings': ingTok, 'title': titleTok}

    def __len__(self):
        return len(self.idxWords)

    def __getitem__(self, index):
      # tuple of input (ingredients) and label (title)
        return (
            torch.tensor(self.idxWords[index]['ings'], device=device),
            torch.tensor(self.idxWords[index]['title'], device=device)
        )

time: 28.7 ms (started: 2022-02-20 11:04:26 +00:00)


In [18]:
class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3

        n_vocab = len(dataset.unique)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

time: 8.78 ms (started: 2022-02-20 09:27:41 +00:00)


In [37]:
def train(dataset, model):
    model.train()

    dataloader = DataLoader(dataset, batch_size=dataset.hyperparams.batchSize)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(dataset.hyperparams.epochs):
        state_h, state_c = model.init_state(dataset.hyperparams.sequenceLength)

        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

    #         y_pred, (state_h, state_c) = model(x, (state_h, state_c))
    #         loss = criterion(y_pred.transpose(1, 2), y)

    #         state_h = state_h.detach()
    #         state_c = state_c.detach()

    #         loss.backward()
    #         optimizer.step()

    #         print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

time: 7.17 ms (started: 2022-02-20 10:32:10 +00:00)


In [20]:
def predict(dataset, model, text, next_words=100):
    model.eval()

    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])

    return words

time: 7.49 ms (started: 2022-02-20 09:27:41 +00:00)


### Execution

In [47]:
titleSet = TitleDataset(hyperParams(), baseFrame)
model = Model(titleSet)

time: 37.7 s (started: 2022-02-20 11:04:30 +00:00)


In [38]:
train(titleSet, model)

time: 2.03 ms (started: 2022-02-20 10:32:13 +00:00)


In [21]:
# print(predict(dataset, model, text='Knock knock. Whos there?'))

time: 749 µs (started: 2022-02-20 09:27:41 +00:00)


In [48]:
for i_batch, sample_batched in enumerate(DataLoader(titleSet, batch_size=titleSet.hyperparams.batchSize)):
    print('hello')

RuntimeError: ignored

time: 82.7 ms (started: 2022-02-20 11:05:08 +00:00)


In [52]:
titleSet[0][0]

tensor([1852,  734,    0, 1679,   23,    0,  218,    0,  104,  175,    0, 2573,
           0,  310,    0,  122,    0,   13,    0,  115,  521, 1493],
       device='cuda:0')

time: 4.93 ms (started: 2022-02-20 11:23:32 +00:00)


In [51]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

TypeError: ignored

time: 9.94 ms (started: 2022-02-20 11:23:00 +00:00)


In [54]:
data = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
data = data.reshape((1, 10, 1))
print(data.shape)

(1, 10, 1)
time: 2.23 ms (started: 2022-02-20 11:45:41 +00:00)
